In [1]:
#installed with pip
from pandas import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
# from urllib.error import HTTPError
from collections import namedtuple

# built-in libraries
import datetime
import re
import sqlite3
import urllib
import csv
import datetime

In [2]:
# Read state summary report for District into Pandas df; filter for needed fields and assign headers
insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                   usecols=[2, 4, 5, 6, 7, 8, 9, 12, 13, 14, 17, 18, 80, 81], encoding="ISO-8859-1")
insp.columns = ["county", "licnum", "sitename", "streetaddy", "cityaddy", "zip",
                "inspnum", "insptype", "inspdispos", "inspdate", "totalvio", "highvio", "licid", "visitid"]
county_sought = 'Marion' # uncomment if particular county sought
insp = insp[(insp.county == county_sought)] # uncomment if particular county sought
insp.sitename = insp.sitename.str.title() #titlecase sitename
insp.streetaddy = insp.streetaddy.str.title() #titlecase street adress
insp.cityaddy = insp.cityaddy.str.title() #titlecase city
insp = insp.applymap(lambda x: str(x).strip() if len(str(x).strip()) else None) #strip whitespace, replace empty vals
insp['visitid'] = insp['visitid'].apply(int) # so it can be filtered against df below

# Read in records from database of earlier reports, create df to filter against new reports in df above.
conn = sqlite3.connect("rinspect.sqlite")
df = pd.read_sql_query("select * from fdinsp;", conn)
df
conn.close()

unique_vals = insp[~insp.visitid.isin(df.visitid)] #filter

In [3]:
# Build a list of URLs to the inspectors detailed reports that will get scraped
result = []
result_for_urls = result # for url list
result_for_list = result.append("NULL") # later into db, space for user-input datetime

# takes LicenseID and VisitID, passes it into the urls for detailed reports later
for index, rows in unique_vals.iterrows():
    visitid = rows['visitid']
    licid = rows['licid']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp? \
        InspVisitID= %s &id= %s" % (visitid, licid)
    urls = urls.replace(' ', '')
    result.append(urls)
urlList = result
urlList.pop(0) # get rid of first "Null" from append above
# len(urlList)

0

In [4]:
# Place data from state summary report df into database
# first, interate through the df to return tuples
var = list(unique_vals.itertuples(index='visitid', name=None))
# populate database table for inspection summmary reports
sqlite_file = 'rinspect.sqlite'
fdinsp_table = 'fdinsp' # table for summary data
id_field = 'visitid' # ID column
time_now = 'now' # column for user-input timestamp; until then = NULL

# connect to database and insert new summary report data
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.executemany('''INSERT OR IGNORE INTO fdinsp (librow, county, licnum, sitename,
              streetaddy, cityaddy, zip, inspnum, insptype, inspdispos,
              inspdate, totalvio, highvio, licid, visitid)
              VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', var)
conn.commit()
conn.close()

In [5]:
# Next step: get data from ispectors observations
# function to scrape inspections from list of urls above, place in data structure
details_dict = {}
def make_details_dict(url):
    inspurl = url
    # extract VisitID number from url; will be pk in database
    visitid.append(inspurl.split("VisitID=")[1].split("&")[0]) 
    # or ...
    # visitid = inspurl[inspurl.rfind("VisitID=") + 1:8]
    html = urlopen(url)
    soup = bs(html.read(), 'lxml')
    details = soup.find_all('font', {'face': 'verdana'})[10:] # relevant text on pages in verdana
    result = []

    site_name = details[0].get_text
    high_num = (details[24].get_text)
    intermed_num = (details[25].get_text)
    basic_num = (details[26].get_text)
    totalvio = (int(details[24].text)) + (int(details[25].text)) + (int(details[26].text))
    # haven't yet figured out to get observations without error if they don't exist in report
    # if (details[32].text):                               
#         observed1 = (details[32].text)
#     else:
#         observed1 = 'NaN'


    details_dict = {
        'visitid' : visitid,
        'restaurant': site_name,
#        'totalvio' : total_num,
#        'observed1': observed1,
#        'observed2': observed2,
#        'observed3': observed3,
#        'observed4': observed4,
#        'observed5': observed5,
#        'observed6': observed6,
#        'observed7': observed7,
#        'observed8': observed8,
#        'observed9': observed9,
#        'observed10': observed10,
#        'observed11': observed11,
#        'observed12': observed12,
    }
    result.append(details_dict)    
        
    return details_dict

# iterate through urls, calling function above to make dictionary
for url in urlList:
    make_details_dict(url)
                          
# need to send this stuff to dict once it's working

In [ ]:
list(details_dict.values())

In [ ]:
# a shorter try for a list
details_list = []
def make_details_list(url):
    inspurl = url
    visitid = inspurl[inspurl.rfind("VisitID=") + 1:8]
    
    html = urlopen(url)
    soup = bs(html.read(), 'lxml')
    details = soup.find_all('font', {'face': 'verdana'})[10:] # relevant text on pages in verdana
    result = []

    for detail in details:
        row = [
        details[0].get('business'),
        details[24].get('visitid'),
        details[25].get('intermed_num'),
        details[26].get('basic_num'),      
        ]    
        row.append(details_list)    
    return details_list    

for url in urlList:
    make_details_list(url)


In [ ]:
# a shorter try for a list
details_list = []
def make_details_list(url):
    inspurl = url
    visitid = inspurl[inspurl.rfind("VisitID=") + 1:8]
    
    html = urlopen(url)
    soup = bs(html.read(), 'lxml')
    details = soup.find_all('font', {'face': 'verdana'})[10:] # relevant text on pages in verdana
    result = []
        
    for detail in details:
        row = [
        details[0].get('business'),
        details[24].get('visitid'),
        details[25].get('intermed_num'),
        details[26].get('basic_num'),      
        if not soup.find(details[32]):
            break
        else:    
            observed1 = details[32].text
        if not soup.find(details[34]):
            break
        else:
            observed2 = details[34].text
        if not soup.find(details[36]):
            break            
        else:
            observed3 = details[36].text
        if not soup.find(details[38]):
           break             
        else:
            observed4 = details[38].text
        if not soup.find(details[40]):
            break
        else:
            observed5 = details[40].text
        if not soup.find(details[42]):
            break
        else:
            observed6 = details[42].text
        if not soup.find(details[44]):
            break
        else:
            observed7 = details[44].text
        if not soup.find(details[46]):
            break
        else:
            observed8 = details[46].text
        if not soup.find(details[48]):
            break
        else:
            observed9 = details[48].text
        if not soup.find(details[50]):
            break
        else:
            observed10 = details[50].text
        if not soup.find(details[52]):
            break
        else:
            observed11 = details[52].text
        if not soup.find(details[54]):
            break
        else:
            observed12 = details[54].text
        ]    
        row.append(details_list)    
    result.append(details_list)
    return details_list    

for url in urlList:
    make_details_list(url)


In [ ]:
sqlite_file = 'rinspect.sqlite'
connector.executemany("insert into sqlite_file values (NULL,?,?,?,?,?)", *row)